In [1]:
from nilmtk import DataSet
import pandas as pd
import numpy as np
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt

# Load UK-DALE dataset
dataset = DataSet("D:/lusip/ukdale.h5/ukdale.h5")
elec = dataset.buildings[1].elec

# Load active power for mains
mains_df = next(elec.mains().load(sample_period=6))[('power', 'active')].to_frame()
mains_df.columns = ['mains']

# Load active power for kettle
kettle_df = next(elec['kettle'].load(sample_period=6))[('power', 'active')].to_frame()
kettle_df.columns = ['kettle']


c:\Users\tanmay\AppData\Local\Programs\Python\Python313\Lib\site-packages\nilmtk\utils.py:509: UserWarning: Found duplicate index. Keeping first value
  warnings.warn("Found duplicate index. Keeping first value")
c:\Users\tanmay\AppData\Local\Programs\Python\Python313\Lib\site-packages\nilmtk\utils.py:501: FutureWarning: 'S' is deprecated and will be removed in a future version, please use 's' instead.
  return fill_method(how(data.resample(rule, **resample_kwargs)))
c:\Users\tanmay\AppData\Local\Programs\Python\Python313\Lib\site-packages\nilmtk\utils.py:501: FutureWarning: 'S' is deprecated and will be removed in a future version, please use 's' instead.
  return fill_method(how(data.resample(rule, **resample_kwargs)))


In [2]:
# Align by timestamp
aligned_df = mains_df.join(kettle_df, how='inner')
aligned_df.dropna(inplace=True)

# Remove kettle peaks > 3000W
aligned_df = aligned_df[aligned_df['kettle'] <= 3000]


In [3]:
# Create and fit scalers
mains_scaler = MinMaxScaler()
kettle_scaler = MinMaxScaler()

aligned_df['mains'] = mains_scaler.fit_transform(aligned_df[['mains']])
aligned_df['kettle'] = kettle_scaler.fit_transform(aligned_df[['kettle']])


In [4]:
def create_sequences(df, window_size):
    X, y = [], []
    for i in range(len(df) - window_size):
        X.append(df['mains'].values[i:i + window_size])
        y.append(df['kettle'].values[i + window_size // 2])  # seq2point
    return np.array(X), np.array(y)

WINDOW_SIZE = 99
X, y = create_sequences(aligned_df, WINDOW_SIZE)

# Reshape for LSTM input
X = X.reshape(-1, WINDOW_SIZE, 1)


In [5]:
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42
)

print("X_train shape:", X_train.shape)
print("X_test shape:", X_test.shape)


X_train shape: (7474748, 99, 1)
X_test shape: (1868688, 99, 1)


In [8]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset
from sklearn.metrics import mean_absolute_error
import optuna
import numpy as np

# 🔧 Define model
class LSTMTransformer(nn.Module):
    def __init__(self, hidden_size, num_heads, ff_dim, num_layers):
        super().__init__()
        self.lstm = nn.LSTM(input_size=1, hidden_size=hidden_size, batch_first=True)
        encoder_layer = nn.TransformerEncoderLayer(
            d_model=hidden_size,
            nhead=num_heads,
            dim_feedforward=ff_dim,
            batch_first=True
        )
        self.transformer = nn.TransformerEncoder(encoder_layer, num_layers=num_layers)
        self.fc = nn.Linear(hidden_size, 1)

    def forward(self, x):
        x, _ = self.lstm(x)
        x = self.transformer(x)
        center = x[:, x.size(1) // 2, :]
        return self.fc(center).squeeze()

# 🎯 Define Optuna objective for kettle
def objective(trial):
    # Sample hyperparameters
    num_heads = trial.suggest_categorical('num_heads', [2, 4, 8])
    hidden_size_choices = [h for h in range(32, 129, 8) if h % num_heads == 0]
    hidden_size = trial.suggest_categorical('hidden_size', hidden_size_choices)
    ff_dim = trial.suggest_int('ff_dim', 64, 256)
    num_layers = trial.suggest_int('num_layers', 1, 3)
    lr = trial.suggest_float('lr', 1e-4, 1e-2, log=True)
    batch_size = trial.suggest_categorical('batch_size', [64, 128, 256])

    # ⚡ Subset for speed
    subset_size = 300_000
    X_sub = X_train[:subset_size]
    y_sub = y_train[:subset_size]

    train_dataset = TensorDataset(
        torch.tensor(X_sub, dtype=torch.float32),
        torch.tensor(y_sub, dtype=torch.float32)
    )
    train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)

    # Initialize model
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    print("CUDA Available:", torch.cuda.is_available())
    print("Device:", torch.cuda.get_device_name(0))
    model = LSTMTransformer(hidden_size, num_heads, ff_dim, num_layers).to(device)

    criterion = nn.MSELoss()
    optimizer = optim.Adam(model.parameters(), lr=lr)

    # 🔁 Train for 10 epochs
    model.train()
    for epoch in range(5):
        for xb, yb in train_loader:
            xb, yb = xb.to(device), yb.to(device)
            optimizer.zero_grad()
            output = model(xb)
            loss = criterion(output, yb)
            loss.backward()
            optimizer.step()

    # 📈 Evaluate
    model.eval()
    predictions, actuals = [], []
    with torch.no_grad():
        for xb, yb in DataLoader(
            TensorDataset(
                torch.tensor(X_test, dtype=torch.float32),
                torch.tensor(y_test, dtype=torch.float32)
            ),
            batch_size=batch_size
        ):
            xb = xb.to(device)
            preds = model(xb).cpu().numpy()
            predictions.extend(preds)
            actuals.extend(yb.numpy())

    # Inverse transform
    preds_inv = kettle_scaler.inverse_transform(np.array(predictions).reshape(-1, 1)).flatten()
    actuals_inv = kettle_scaler.inverse_transform(np.array(actuals).reshape(-1, 1)).flatten()

    return mean_absolute_error(actuals_inv, preds_inv)


In [9]:

# Create study
study = optuna.create_study(direction='minimize')
# Run tuning
study.optimize(objective, n_trials=10)
# Show best results
print("\n✅ Best MAE:", study.best_value)
print("✅ Best Hyperparameters:", study.best_params)


[I 2025-07-06 16:59:34,857] A new study created in memory with name: no-name-059c45ca-8f86-463e-be6d-f2fe761e702a


CUDA Available: True
Device: NVIDIA GeForce RTX 3050 Laptop GPU


[I 2025-07-06 17:05:18,466] Trial 0 finished with value: 17.005935668945312 and parameters: {'num_heads': 2, 'hidden_size': 56, 'ff_dim': 183, 'num_layers': 3, 'lr': 0.004499546765618546, 'batch_size': 128}. Best is trial 0 with value: 17.005935668945312.


CUDA Available: True
Device: NVIDIA GeForce RTX 3050 Laptop GPU


[I 2025-07-06 17:13:24,769] Trial 1 finished with value: 8.636940002441406 and parameters: {'num_heads': 4, 'hidden_size': 72, 'ff_dim': 208, 'num_layers': 3, 'lr': 0.00011090132629828084, 'batch_size': 256}. Best is trial 1 with value: 8.636940002441406.


...

CUDA Available: True
Device: NVIDIA GeForce RTX 3050 Laptop GPU


[I 2025-07-06 17:55:33,451] Trial 8 finished with value: 30.51527976989746 and parameters: {'num_heads': 2, 'hidden_size': 96, 'ff_dim': 138, 'num_layers': 1, 'lr': 0.00030033187435503074, 'batch_size': 256}. Best is trial 1 with value: 8.636940002441406.


CUDA Available: True
Device: NVIDIA GeForce RTX 3050 Laptop GPU


[I 2025-07-06 18:03:36,557] Trial 9 finished with value: 18.654237747192383 and parameters: {'num_heads': 2, 'hidden_size': 88, 'ff_dim': 114, 'num_layers': 3, 'lr': 0.005981972981678149, 'batch_size': 64}. Best is trial 1 with value: 8.636940002441406.



✅ Best MAE: 8.636940002441406
✅ Best Hyperparameters: {'num_heads': 4, 'hidden_size': 72, 'ff_dim': 208, 'num_layers': 3, 'lr': 0.00011090132629828084, 'batch_size': 256}
